In [4]:
import pandas as pd
import instance_generation

In [5]:
import glob
glob.glob('./../../Data/parc_features/*')

['./../../Data/parc_features\\parc_dev_features.tsv',
 './../../Data/parc_features\\parc_test_features.tsv',
 './../../Data/parc_features\\parc_test_predicted_features.tsv',
 './../../Data/parc_features\\parc_train_features.tsv']

In [6]:
filepath = './../../Data/parc_features/parc_dev_features.tsv'

In [7]:
def create_instance_list(list_of_tuples,df):
    main_list = list()

    # Loop through instances meaning content and source span indices
    for instance in pair_list:
        # initiate instance_list and index_list
        attribution_indices = list()

        source, content = instance
        b_source, e_source = source
        b_content, e_content = content

        # For index of source
        for index in range(b_source, e_source+1):  
            attribution_indices.append(index)
        # for index of content
        for index in range(b_content, e_content+1):
            attribution_indices.append(index)
            
        # Find gap indices
        if b_source < b_content: 
            # If content follows source, gap is between last token of source and first of content
            gap_indices = [e_source, b_content]
        else:
            # If source follows content, then visa versa
            gap_indices = [e_content, b_source]

        list_tuple = (attribution_indices, gap_indices)
        main_list.append(list_tuple)
        
    return main_list

def find_sc_dist(df, main_list):
    
    df['s/c_distance'] = 'X'
    
    for list_tupel in main_list:
        instance_list = list_tupel[0]
        index_list = list_tupel[1]
        
        if instance_list[0] < instance_list[1]:
            distance = instance_list[1] - instance_list[0]
        else:
            distance = instance_list[0] - instance_list[1]

        df.loc[index_list,'s/c_distance'] = distance

    #return df
    
def find_num_conts_between(df, main_list):
    
    df['num_conts_between'] = 'X'
    
    for list_tupel in main_list:
        instance_list = list_tupel[0]
        index_list = list_tupel[1]
        
        counter = 0
        for index in list(range(instance_list[0], instance_list[1])):
            if df.loc[index,'content_label_gold'] == 'B':
                counter += 1
                
        df.loc[index_list,'num_conts_between'] = counter
        
    #return df
    
def find_num_sources_between(df, main_list):
    
    df['num_sources_between'] = 'X'
    
    for list_tupel in main_list:
        instance_list = list_tupel[0]
        index_list = list_tupel[1]
        
        counter = 0
        for index in list(range(instance_list[0], instance_list[1])):
            # TODO: Q changed from candidate_source to candidate_source_label. Check with Ellie if correct
            if df.loc[index,'candidate_source_label'] == 'B':
                counter += 1
                
        df.loc[index_list,'num_sources_between'] = counter
        
    #return df
    
def run_dist_feats(list_of_tuples,df):
    
    main_list = create_instance_list(list_of_tuples,df)
    find_sc_dist(df, main_list)
    find_num_conts_between(df, main_list)
    find_num_sources_between(df, main_list)
    
    return df




In [8]:
df = pd.read_csv(filepath, delimiter='\t', index_col=0)
df = df[:100]

C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
instance_output = instance_generation.collect_instances_main(df)

In [10]:
gold_label_list = list()
pair_list = list()
for instance_list in instance_output:
    gold_label_list.append(instance_list[2])
    pair_tupel = (instance_list[0], instance_list[1])
    pair_list.append(pair_tupel)

print(gold_label_list[:4])
print(pair_list[:4])

[0, 0, 1, 1]
[((41, 43), (40, 58)), ((53, 54), (40, 58)), ((62, 63), (40, 58)), ((64, 66), (40, 58))]


In [12]:
main_list = create_instance_list(pair_list, df)

In [22]:
def find_sc_dist(df, main_list):
    
    df['s/c_distance'] = 'X'
    
    for index_list, instance_list  in main_list:
        print(instance_list)
        if instance_list[0] < instance_list[1]:
            print('a')
            distance = instance_list[1] - instance_list[0]
        else:
            print('b')
            distance = instance_list[0] - instance_list[1]
        df.loc[index_list,'s/c_distance'] = distance
        
find_sc_dist(df, main_list)

[58, 41]
b
[58, 53]
b
[58, 62]
a
[58, 64]
a
[58, 68]
a
[58, 70]
a
[58, 73]
a
[77, 87]
a
[79, 87]
a
[85, 87]
a
[89, 97]
a


In [20]:
df

POS  dependency_head dependency_label  doc_token_number        lemma  \
0    DT                2              det                 1          the   
1    NN                4        nmod:poss                 2      economy   
2   POS                2             case                 3           's   
3    NN                7       nsubj:pass                 4  temperature   
4    MD                7              aux                 5         will   
5    VB                7         aux:pass                 6           be   
6   VBN                0             root                 7         take   
7    IN               11             case                 8         from   
8    JJ               11             amod                 9      several   
9    NN               11         compound                10      vantage   
10  NNS                7              obl                11        point   
11   DT               13              det                12         this   
12   NN                7         obl:tmod                13         week   
13    ,                7            punct                14            ,   
14   IN               16             case                15         with   
15  NNS                7              obl                16      reading   
16   IN               18             case                17           on   
17   NN               16             nmod                18        trade   
18    ,               20            punct                19            ,   
19   NN               18             conj                20       output   
20    ,               22            punct                21            ,   
21   NN               18             conj                22      housing   
22   CC               24               cc                23          and   
23   NN               18             conj                24    inflation   
24    .                7            punct                25            .   
25   DT                4              det                26          the   
26  RBS                3           advmod                27         most   
27   JJ                4             amod                28  troublesome   
28   NN               11            nsubj                29       report   
29   MD               11              aux                30          may   
..  ...              ...              ...               ...          ...   
70  NNP               32         compound                71  McGraw-Hill   
71  NNP               29             nmod                72         Inc.   
72    ,               26            punct                73            ,   
73  NNP               35         compound                74          New   
74  NNP               26            appos                75         York   
75    .                5            punct                76            .   
76  NNP                3        nmod:poss                77     Thursday   
77  POS                1             case                78           's   
78   NN               11       nsubj:pass                79       report   
79   IN                9             case                80           on   
80   DT                9              det                81          the   
81  NNP                9         compound                82    September   
82   NN                8         compound                83     consumer   
83   NN                9         compound                84        price   
84   NN                3             nmod                85        index   
85  VBZ               11         aux:pass                86           be   
86  VBN                0             root                87       expect   
87   TO               13             mark                88           to   
88   VB               11            xcomp                89         rise   
89    ,               13            punct                90            ,   
90   IN               18             mark           

In [16]:

#find_num_conts_between(df, main_list)
#find_num_sources_between(df, main_list)



([41, 42, 43, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 41])
([53, 54, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 53])
([62, 63, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 62])
([64, 65, 66, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 64])
([68, 69, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 68])
([70, 71, 72, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 70])
([73, 74, 75, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [58, 73])
([76, 77, 87, 88, 89], [77, 87])
([78, 79, 87, 88, 89], [79, 87])
([81, 82, 83, 84, 85, 87, 88, 89], [85, 87])
([97, 98, 99, 87, 88, 89], [89, 97])


In [56]:
def find_sc_dist(df, main_list):
    
    df['s/c_distance'] = 'X'

    for instance in main_list:
        # Unpack instances
        source_indices, content_indices = instance
        
        # Calculate distance by subracting highest start index from lowest
        if source_indices[0] > content_indices[0]:
            distance = source_indices[0] - content_indices[0]
        else:
            distance = content_indices[0] - source_indices[0]
        
        # Update df
        df.loc[index_list,'s/c_distance'] = distance
